In [7]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

In [8]:
llm = OpenAI(temperature=0)

In [9]:
tools = load_tools(["serpapi", "llm-math"], llm=llm)

## Using LCEL

In [10]:
from langchain.tools.render import render_text_description
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain import hub

In [11]:
prompt = hub.pull("hwchase17/react")
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

In [12]:
llm_with_stop = llm.bind(stop=["\nObservation"])

In [13]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

In [14]:
from langchain.agents import AgentExecutor

In [15]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [17]:
agent_executor.invoke(
    {
        "input": "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
    }
)



> Entering new AgentExecutor chain...


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None)).


 I need to find out who Leo DiCaprio's girlfriend is and then calculate her age raised to the 0.43 power.
Action: Search
Action Input: "Leo DiCaprio girlfriend"["DiCaprio gets handsy with his new model GF, Vittoria Ceretti, but he isn't the one feeling her up here."] I need to find out Vittoria Ceretti's age.
Action: Search
Action Input: "Vittoria Ceretti age"25 years I need to calculate 25 raised to the 0.43 power.
Action: Calculator
Action Input: 25^0.43Answer: 3.991298452658078 I now know the final answer.
Final Answer: Leo DiCaprio's girlfriend is Vittoria Ceretti and her current age raised to the 0.43 power is 3.991298452658078.

> Finished chain.


{'input': "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?",
 'output': "Leo DiCaprio's girlfriend is Vittoria Ceretti and her current age raised to the 0.43 power is 3.991298452658078."}

In [18]:
agent_executor = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [19]:
agent_executor.invoke(
    {
        "input": "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
    }
)



> Entering new AgentExecutor chain...
 I need to find out who Leo DiCaprio's girlfriend is and then calculate her age raised to the 0.43 power.
Action: Search
Action Input: "Leo DiCaprio girlfriend"
Observation: ["DiCaprio gets handsy with his new model GF, Vittoria Ceretti, but he isn't the one feeling her up here."]
Thought: I need to find out Vittoria Ceretti's age.
Action: Search
Action Input: "Vittoria Ceretti age"
Observation: 25 years
Thought: I need to calculate 25 raised to the 0.43 power.
Action: Calculator
Action Input: 25^0.43
Observation: Answer: 3.991298452658078
Thought: I now know the final answer.
Final Answer: Leo DiCaprio's girlfriend is Vittoria Ceretti and her current age raised to the 0.43 power is 3.991298452658078.

> Finished chain.


{'input': "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?",
 'output': "Leo DiCaprio's girlfriend is Vittoria Ceretti and her current age raised to the 0.43 power is 3.991298452658078."}

In [29]:
from langchain.chat_models import ChatOpenAI

In [30]:
chat_model = ChatOpenAI(temperature=0)

In [31]:
prompt = hub.pull("hwchase17/react-json")
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

In [32]:
chat_model_with_stop = chat_model.bind(stop=["\nObservation"])

In [33]:
from langchain.agents.output_parsers import ReActJsonSingleInputOutputParser

In [34]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | chat_model_with_stop
    | ReActJsonSingleInputOutputParser()
)

In [35]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [36]:
agent_executor.invoke(
    {
        "input": "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
    }
)



> Entering new AgentExecutor chain...
Thought: I need to find out who is Leo DiCaprio's girlfriend and then calculate her current age raised to the 0.43 power.

Action:
```
{
  "action": "Search",
  "action_input": "Leo DiCaprio girlfriend"
}
```
["DiCaprio gets handsy with his new model GF, Vittoria Ceretti, but he isn't the one feeling her up here."]Based on the search results, it seems that Leo DiCaprio's current girlfriend is Vittoria Ceretti. Now I need to calculate her current age raised to the 0.43 power.

Action:
```
{
  "action": "Calculator",
  "action_input": "age ^ 0.43"
}
```


ValueError: unknown format from LLM: This question is not valid for the numexpr library, as it requires a numerical value for the variable "age".